In [104]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os

In [105]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [132]:
import mujoco_py
mjcModel  = mujoco_py.load_model_from_path("../../../myosuite/envs/myo/assets/hand/myo_hand_pose.mjb")
sim_mjc = mujoco_py.MjSim(mjcModel)
print("Muscles:")
for i in range(len(mjcModel.actuator_names)):
    print([i,mjcModel.actuator_names[i]])
# print(mjcModel.joint_names)

print("\nJoints:")
for i in range(len(mjcModel.joint_names)):
    print([i,mjcModel.joint_names[i]])

viewer = mujoco_py.MjRenderContextOffscreen(sim_mjc, 0)


musc_fe = [mjcModel.actuator_names.index('FDP2'),mjcModel.actuator_names.index('EDC2')]
L_range = round(1/mjcModel.opt.timestep)
skip_frame = 50
sim_mjc.reset()

frames_sim = []
for iter_n in range(3):
    print("iteration: "+str(iter_n))
    res_sim = []
    for rp in range(2): #alternate between flexor and extensor
        for s in range(L_range):
            if not(s%skip_frame):
                viewer.render(400, 400, 2)
                frames_sim.append(np.asarray(viewer.read_pixels(400, 400, depth=False)[::-1, :, :], dtype=np.uint8))

            for i in range(len(sim_mjc.data.ctrl)):
                sim_mjc.data.ctrl[i] = 0

            act_val = 1 # maximum muscle activation
            if rp==0:
                sim_mjc.data.ctrl[musc_fe[0]] = act_val
                sim_mjc.data.ctrl[musc_fe[1]] = 0
            else:
                sim_mjc.data.ctrl[musc_fe[1]] = act_val
                sim_mjc.data.ctrl[musc_fe[0]] = 0                        
            sim_mjc.step()

os.makedirs('videos', exist_ok=True)
# make a local copy
skvideo.io.vwrite('videos/MyoSuite.mp4', np.asarray(frames_sim),outputdict={"-pix_fmt": "yuv420p"})

# show in the notebook
show_video('videos/MyoSuite.mp4')

Muscles:
[0, 'ECRL']
[1, 'ECRB']
[2, 'ECU']
[3, 'FCR']
[4, 'FCU']
[5, 'PL']
[6, 'PT']
[7, 'PQ']
[8, 'FDS5']
[9, 'FDS4']
[10, 'FDS3']
[11, 'FDS2']
[12, 'FDP5']
[13, 'FDP4']
[14, 'FDP3']
[15, 'FDP2']
[16, 'EDC5']
[17, 'EDC4']
[18, 'EDC3']
[19, 'EDC2']
[20, 'EDM']
[21, 'EIP']
[22, 'EPL']
[23, 'EPB']
[24, 'FPL']
[25, 'APL']
[26, 'OP']
[27, 'RI2']
[28, 'LU_RB2']
[29, 'UI_UB2']
[30, 'RI3']
[31, 'LU_RB3']
[32, 'UI_UB3']
[33, 'RI4']
[34, 'LU_RB4']
[35, 'UI_UB4']
[36, 'RI5']
[37, 'LU_RB5']
[38, 'UI_UB5']

Joints:
[0, 'pro_sup']
[1, 'deviation']
[2, 'flexion']
[3, 'cmc_abduction']
[4, 'cmc_flexion']
[5, 'mp_flexion']
[6, 'ip_flexion']
[7, 'mcp2_flexion']
[8, 'mcp2_abduction']
[9, 'pm2_flexion']
[10, 'md2_flexion']
[11, 'mcp3_flexion']
[12, 'mcp3_abduction']
[13, 'pm3_flexion']
[14, 'md3_flexion']
[15, 'mcp4_flexion']
[16, 'mcp4_abduction']
[17, 'pm4_flexion']
[18, 'md4_flexion']
[19, 'mcp5_flexion']
[20, 'mcp5_abduction']
[21, 'pm5_flexion']
[22, 'md5_flexion']
Creating window glfw
iteration: 0
